In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests

from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from os import path

### 0. Functions

In [2]:
def getTickers():
    """Returns the tickers for all the S&P500 companies using the Wikipedia page
    Outputs: 
        tickers - list of tickers for every company in the S&P500
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find("table") # tickers are contained in a table
    tickers = []
    for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols:
                tickers.append(cols[0].text.strip())
    return tickers

In [3]:
def convert_options_to_dict(df, expiration_date, underlying_price):
    """ Converts a DataFrame of options data into a list of dictionaries with specific fields.
    Args:
        df (pd.DataFrame): DataFrame containing options data.
        expiration_date (str): Expiration date in 'YYYY-MM-DD' format.
        underlying_price (float): Current price of the underlying asset.
        Returns:
        list: List of dictionaries with selected fields from the DataFrame.
    """
    expiration_datetime = datetime.strptime(expiration_date, '%Y-%m-%d')
    df = df.assign(expiration=expiration_datetime, price=underlying_price)
    df['lastTradeDate'] = df['lastTradeDate'].apply(lambda x: x.replace(tzinfo=None))

    return df.to_dict(orient='records')

In [4]:
def get_options_data(ticker):
    """Returns the options data for a given ticker
    Inputs:
        ticker - string, the ticker symbol of the stock
    Outputs:
        calls - list of dictionaries containing call options data
        puts - list of dictionaries containing put options data
    """
    calls = []
    puts = []

    stock = yf.Ticker(ticker)
    for expiration_date in stock.options:
        opt = stock.option_chain(expiration_date)
        opt_calls = opt.calls
        opt_puts = opt.puts
        underlying_price = opt.underlying['regularMarketPrice']
        calls.append(convert_options_to_dict(opt_calls, expiration_date, underlying_price))
        puts.append(convert_options_to_dict(opt_puts, expiration_date, underlying_price))

    return calls, puts

In [5]:
def generate_options_df(options):
    """
    Generates a dataframe from the options data in the format returned by get_options_data.
    """
    options = [item for sublist in options for item in sublist]
    df_opts = pd.DataFrame.from_records(options)
    df_opts['duration'] = df_opts['expiration'] - df_opts['lastTradeDate']
    df_opts['duration'] = df_opts['duration'].apply(lambda x: x.days)
    df_opts['remaining'] = df_opts['expiration'].apply(lambda x: (x - datetime.now()).days)

    return df_opts

In [6]:
def get_all_options_data(tickers):
    """Fetches options data for a list of tickers.
    Args:
        tickers (list): List of ticker symbols.
    Returns:
        tuple: Two DataFrames, one for call options and one for put options.
    """

    # with ThreadPoolExecutor(max_workers=100) as p:
    #      results = p.map(get_options_data, tickers)

    results = []
    for ticker in tickers:
        print(f"Fetching data for {ticker}")
        try:
            result = get_options_data(ticker)
            results.append(result)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    calls = []
    puts = []

    for result in results:
        calls.extend(result[0])
        puts.extend(result[1])

    df_calls = generate_options_df(calls)
    df_puts = generate_options_df(puts)

    return df_calls, df_puts

### 1. Scrapping data

In [7]:
# Get today date as year_month_day string
today = datetime.now()
today_str = today.strftime('%Y_%m_%d')

# Use the date to create the filename
tickers = getTickers()
df_calls, df_puts = get_all_options_data(tickers)

# Save the data to CSV files
df_calls.to_csv(f'../data/raw/sp500_calls_{today_str}.csv', index=False)
df_puts.to_csv(f'../data/raw/sp500_puts_{today_str}.csv', index=False)

Fetching data for MMM
Fetching data for AOS
Fetching data for ABT
Fetching data for ABBV
Fetching data for ACN
Fetching data for ADBE
Fetching data for AMD
Fetching data for AES
Fetching data for AFL
Fetching data for A
Fetching data for APD
Fetching data for ABNB
Fetching data for AKAM
Fetching data for ALB
Fetching data for ARE
Fetching data for ALGN
Fetching data for ALLE
Fetching data for LNT
Fetching data for ALL
Fetching data for GOOGL
Fetching data for GOOG
Fetching data for MO
Fetching data for AMZN
Fetching data for AMCR
Fetching data for AEE
Fetching data for AEP
Fetching data for AXP
Fetching data for AIG
Fetching data for AMT
Fetching data for AWK
Fetching data for AMP
Fetching data for AME
Fetching data for AMGN
Fetching data for APH
Fetching data for ADI
Fetching data for ANSS
Fetching data for AON
Fetching data for APA
Fetching data for APO
Fetching data for AAPL
Fetching data for AMAT
Fetching data for APTV
Fetching data for ACGL
Fetching data for ADM
Fetching data for 

In [8]:
df_calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration,price,duration,remaining
0,MMM250606C00120000,2025-05-30 19:38:26,120.0,28.69,24.95,26.75,0.0,0.0,2.0,1.0,1.742189,True,REGULAR,USD,2025-06-06,146.29,6,0
1,MMM250606C00123000,2025-06-04 15:06:09,123.0,24.90,21.95,23.25,0.0,0.0,32.0,32.0,0.000010,True,REGULAR,USD,2025-06-06,146.29,1,0
2,MMM250606C00124000,2025-05-07 18:14:01,124.0,14.88,21.55,22.55,0.0,0.0,NaN,0.0,1.335941,True,REGULAR,USD,2025-06-06,146.29,29,0
3,MMM250606C00125000,2025-06-02 13:46:08,125.0,20.20,20.75,21.45,0.0,0.0,2.0,26.0,1.169926,True,REGULAR,USD,2025-06-06,146.29,3,0
4,MMM250606C00127000,2025-05-01 14:32:00,127.0,13.76,20.60,22.80,0.0,0.0,NaN,0.0,2.219243,True,REGULAR,USD,2025-06-06,146.29,35,0
